In [ ]:
language = 'pt'

In [ ]:
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# Código JavaScript para gravar áudio do usuário usando a "MediaStream Recording API"
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec * 1000))
  audio = b64decode(js_result.split(',')[1])
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  return f'/content/{file_name}'

print('Ouvindo...\n')
record_file = record()

display(Audio(record_file, autoplay=False))

In [ ]:

import whisper

model = whisper.load_model("base")
result = model.transcribe(record_file, fp16=False, language='en',task='translate')
transcription = result["text"]
print(transcription)

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'SUA_CHAVE'

In [ ]:
import openai
import os

client = openai.OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ { "role": "user", "content": transcription } ]
)

chatgpt_response = response.choices[0].message.content
print(chatgpt_response)

In [ ]:
from gtts import  gTTS

gtts_object = gTTS(text=chatgpt_response, lang='en', slow=False)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)

display(Audio(response_audio, autoplay=True))